In [1]:
import glob
import numpy as np
import re
import os

In [10]:
def filePaths(path):
    return glob.glob(path)

In [2]:
# Doc noi dung cua tung file text
# tach tung tu trong file text
# Xay dung tap 'dictionary' chua danh sach cac tu
def parseWord(file_paths):
    lst_contents = []
    dictionary = set()

    for path in file_paths:
        with open(path,'r') as file:
            string = file.read()
            # Loai bo ki tu dac biet
            content = set(re.sub('[^\w\s\/]','',string).split())
            lst_contents.append(content)
            dictionary.update(content)
    dictionary = list(dictionary)
    return (dictionary, lst_contents)

In [3]:
# Buoc 2: Xây dụng inverted files:
def inverted_files(words, texts):
    inv_files = dict()
    for index1, word in enumerate(words):
        inv_files[word] = set()
        for index2, text in enumerate(texts):
            if word in text: 
                inv_files[word].add(index2)
    return inv_files


In [ ]:
# Buoc 3: Xay dung

In [17]:
data = 'datas/*.txt'
files_path = filePaths(data)
dictionary,lst_content = parseWord(files_path)
inv_files = inverted_files(dictionary, lst_content)
print(inv_files)

{'trau': {3}, 'Bàu': {3}, 'Long': {3}, '139': {3}, 'nướcNhững': {3}, 'tha': {3}, 'Hoạt': {3}, '23100': {3}, 'sắt': {3}, 'lây': {3}, 'KiếmHôm': {3}, '51000': {3}, 'tốtTrả': {3}, 'hỏng': {3}, 'mè': {3}, 'theoCòn': {3}, 'ninh': {3}, 'mẫu': {3}, 'khácThứ': {3}, 'chốt': {3}, '1/7/202031/12/2025': {3}, 'Hình': {3}, 'lắng': {3}, 'đục': {3}, 'giòn': {3}, 'A': {3}, 'Cầm': {3}, 'tấcỞ': {3}, 'Hơn': {3}, 'bạo': {3}, 'chếch': {3}, 'lyTheo': {3}, 'karaoke': {0, 1, 2, 3}, 'đét': {3}, 'đồngBà': {3}, '2017Hữu': {3}, '396': {3}, 'Thu': {3}, 'GDP': {3}, '478': {3}, 'tàngNhững': {3}, 'nấng': {3}, 'Thăng': {3}, 'cẩu': {3}, 'lớnÔng': {3}, 'giớiVề': {3}, 'bà': {3}, 'lờ': {3}, 'tài': {3}, 'việnBa': {3}, 'Bác': {3}, 'kẹo': {3}, 'điềm': {3}, 'Khơmer': {3}, '24800': {3}, 'kẽm': {3}, 'top': {3}, 'Vân': {3}, 'Realtime': {3}, '1967': {3}, 'mĐoạn': {3}, 'Lanka': {3}, '68': {3}, '20112016': {3}, 'nhụt': {3}, 'Lời': {3}, 'ôtôCác': {3}, 'Giương': {3}, '594': {3}, 'vmapvn': {3}, '12h05': {3}, 'Indonesia': {3}, 'nhauTuy'

In [7]:
"""# Buoc 2: xay dung ma tran Term-Document
def termMattrix(words,texts):
    termDoc = np.zeros((len(words),len(texts)))
    for index1, word in enumerate(words):
        for index2, text in enumerate(texts):
            if word in text:
                #print('Word: {} in text: {}'.format(content1,index2+1))
                termDoc[index1,index2] = 1
    return termDoc.astype('uint8').astype('str')"""

"# Buoc 2: xay dung ma tran Term-Document\ndef termMattrix(words,texts):\n    termDoc = np.zeros((len(words),len(texts)))\n    for index1, word in enumerate(words):\n        for index2, text in enumerate(texts):\n            if word in text:\n                #print('Word: {} in text: {}'.format(content1,index2+1))\n                termDoc[index1,index2] = 1\n    return termDoc.astype('uint8').astype('str')"

In [18]:
# Buoc 3: Xac dinh cau truy van
def detQuer(truyVan, words, termDoc):
    #truyVan = "'Trump' and 'Biden'"
    pattern1 = "'(\w+)"
    # Lay Logic
    pattern2 = '([oOAaxXnN])\w+\s'
    terms = re.findall(pattern1, truyVan)
    logic = re.findall(pattern2,truyVan)
    logic = [s.capitalize() for s in logic]
    clauses = []
    print(logic)
    print(terms)
    for term in terms:
        try:
            index = words.index(term)
            query = termDoc[index,:]
            clauses.append(query)
            #print(query)
        except:
            print(f'Error! The Word \'{term}\' is not in the query')
            clauses.append(np.zeros((1,len(file_paths))).astype('uint8').astype('str')[0])
    return clauses, logic
    #print(clauses)

In [5]:
# Buoc 4: Xay Dung Luan Ly
def Logic(clauses, logic):
    default = clauses[0].copy()
    index = 1
    iLog = 0
    stop = len(clauses)
    length = len(default)
    dic = {'A': 'and', 'O': 'or', 'X': '^'}
    while True:
        if index == stop:
            break
        else:
            try:
                front = logic[iLog+1]
            except:
                front = ''
            #print(front)
            logi = dic[logic[iLog]]
            term = clauses[index]
            if front == 'N':
                for i in range(length):
                    string = default[i] + " "+ logi+" not " + term[i]
                    #print(string)
                    default[i] = int(eval(string))
                iLog+=2
            else:
                for i in range(length):
                    #print(logi)
                    string = default[i] + " "+ logi+" " + term[i]
                    #print(string)
                    default[i] = int(eval(string))
                #print(eval(string))
                iLog+=1
            index+=1
    return default

In [6]:
def Query(clause,file):
    s = ''
    for i in range(len(clause)):
        if (clause[i] == '1'):
            with open(file[i],'r') as f:
                string = f.read()
                s+=f'Văn bản {i+1}: \n{string}\n'
    return s

In [7]:
def TruyVan(tv, data):
    file_paths = glob.glob(data)
    words, texts = parseWord(file_paths)
    termDoc = termMattrix(words,texts)
    clauses, logic = detQuer(tv, words, termDoc)
    newClause = Logic(clauses, logic)
    query = Query(newClause, file_paths)
    print(query)

In [20]:
def analyze_Word(word):
    pattern = '[^\w\s\=/%-]'
    truyVan = list(set(re.sub(pattern,'',word).split()))
    #truyVan = word.split()
    for i in range(len(truyVan)):
        truyVan[i] = "'"+truyVan[i]+"'"
    return ' and '.join(truyVan)

In [22]:
word = "2020-2021"
tv = analyze_Word(word)
#data = 'data/*.txt'
#TruyVan(tv, data)
print(tv)

'2020-2021'


In [94]:
import pandas as pd
help(pd.DataFrame)

Help on class DataFrame in module pandas.core.frame:

class DataFrame(pandas.core.generic.NDFrame)
 |  DataFrame(data=None, index: Union[Collection, NoneType] = None, columns: Union[Collection, NoneType] = None, dtype: Union[str, numpy.dtype, ForwardRef('ExtensionDtype'), NoneType] = None, copy: bool = False)
 |  
 |  Two-dimensional, size-mutable, potentially heterogeneous tabular data.
 |  
 |  Data structure also contains labeled axes (rows and columns).
 |  Arithmetic operations align on both row and column labels. Can be
 |  thought of as a dict-like container for Series objects. The primary
 |  pandas data structure.
 |  
 |  Parameters
 |  ----------
 |  data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
 |      Dict can contain Series, arrays, constants, or list-like objects.
 |  
 |      .. versionchanged:: 0.23.0
 |         If data is a dict, column order follows insertion-order for
 |         Python 3.6 and later.
 |  
 |      .. versionchanged:: 0.25.0

In [1]:
a = {1,2,4,3}
b = {1,2,3,4}
str(a)

'{1, 2, 3, 4}'

In [2]:
eval(str(a) + ' and ' + str(b))

{1, 2, 3, 4}